This program is to train a model on a sample of dmoz datasets to classify a webpage. This is developed for POC purpose. Lots of work to be done to traing complete training data.

In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dmoz-bert-multiclass-web-classification-dataset/dmoz_full_toplevel_imbalanced.tsv


In [36]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import train_test_split

In [37]:
!pip install --no-cache fastai==1.0.51

In [38]:
from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

In [39]:
!pip install pytorch-pretrained-bert

In [40]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

config = Config(
    testing=False,
    bert_model_name="bert-base-uncased",
    max_lr=3e-5,
    epochs=4,
    use_fp16=True,
    bs=32,
    discriminative=False,
    max_seq_len=256,
)

In [41]:
from pytorch_pretrained_bert import BertTokenizer
bert_tok = BertTokenizer.from_pretrained(
    config.bert_model_name,
)

In [42]:
def _join_texts(texts:Collection[str], mark_fields:bool=False, sos_token:Optional[str]=BOS):
    """Borrowed from fast.ai source"""
    if not isinstance(texts, np.ndarray): texts = np.array(texts)
    if is1d(texts): texts = texts[:,None]
    df = pd.DataFrame({i:texts[:,i] for i in range(texts.shape[1])})
    text_col = f'{FLD} {1} ' + df[0].astype(str) if mark_fields else df[0].astype(str)
    if sos_token is not None: text_col = f"{sos_token} " + text_col
    for i in range(1,len(df.columns)):
        #text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df[i]
        text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df[i].astype(str)
    return text_col.values

In [43]:
class FastAiBertTokenizer(BaseTokenizer):
    """Wrapper around BertTokenizer to be compatible with fast.ai"""
    def __init__(self, tokenizer: BertTokenizer, max_seq_len: int=128, **kwargs):
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length"""
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]

In [44]:
DATA_ROOT=Path('/kaggle/input/dmoz-bert-multiclass-web-classification-dataset/')
df = pd.read_csv(DATA_ROOT / "dmoz_full_toplevel_imbalanced.tsv", sep='\t')

# drop some columns due to memory limitation of colab's resources
df = df.drop(columns=["externalpage_md5"])
df = df.drop(columns=["ages"])
df = df.drop(columns=["mediadate"])
df = df.drop(columns=["priority"])
df = df.drop(columns=["resource_md5"])
df = df.drop(columns=["language"])
df = df.drop(columns=["language_script"])
df = df.drop(columns=["language_iso6391"])
df = df.drop(columns=["language_confidence"])
df = df.drop(columns=["resource"])
df = df.drop(columns=["type"])
df = df.drop(columns=["topic"])
df = df.drop(columns=["topic_parent"])
try:
  df.drop(['Unnamed: 0'],axis=1,inplace=True)
except:
  pass

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
print(df['topic_main'].value_counts())

Society       223890
Business      216443
Arts          207513
Computers      98341
Science        96678
Sports         96376
Recreation     88626
Shopping       72553
Reference      62232
Health         57861
Games          28388
Home           20199
News           10365
Name: topic_main, dtype: int64


In [46]:
print(df.head())

                                        externalpage  \
0                                http://www.awn.com/   
1                                http://www.awn.com/   
2                        http://animation.about.com/   
3                          http://www.toonhound.com/   
4  http://www.digitalmediafx.com/Features/animati...   

                                        title  \
0                     Animation World Network   
1                     Animation World Network   
2                  About.com: Animation Guide   
3                                   Toonhound   
4  Digital Media FX: The History of Animation   

                                         description   catid topic_main  
0  Provides information resources to the internat...  423945       Arts  
1  Provides information resources to the internat...  374370  Computers  
2  Keep up with developments in online animation ...  423945       Arts  
3  British cartoon, animation and comic strip cre...  423945       Arts 

In [47]:
import imblearn
from imblearn.over_sampling import RandomOverSampler

y = df['topic_main']
X = df
# X = df.drop('topic_main', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)

ros = RandomOverSampler()
X_ros, y_ros = ros.fit_sample(X_train, y_train)

# print(X_ros.shape[0] - X.shape[0], 'new random picked points')
# print(pd.Series(y_ros).value_counts()/len(y_ros))

X_resampled = pd.DataFrame(X_ros, columns=X.columns)

print("\nX_resampled:\n")
print(X_resampled.head())
print(X_resampled.shape)

print("\ny_ros:\n")
print(y_ros.head())
print(y_ros.shape)



X_train:

                                              externalpage  \
966684                 http://www.fourdir.com/hisdatsa.htm   
724875                           http://www.coastwalk.org/   
819141                   http://en.wikipedia.org/wiki/Newt   
895150   http://quod.lib.umich.edu/cgi/t/text/pageviewe...   
1264465                       http://cafmarseille.free.fr/   

                                          title  \
966684       Four Directions Institute: Hidatsa   
724875                 California Coastal Trail   
819141                          Wikipedia: Newt   
895150            The St. Louis Bridge Reviewed   
1264465  Club alpin français Marseille Provence   

                                               description   catid  topic_main  
966684                Tribal profile, timeline, and links.  997092     Society  
724875   Points of historical interest, maps, and obser...  958454  Recreation  
819141   Encyclopedia article on these amphibians of th...  999653

In [48]:
#before oversampling
print(pd.Series(y_train).value_counts()/len(y_train))

Society       0.175195
Business      0.169213
Arts          0.162111
Computers     0.076766
Sports        0.075406
Science       0.075387
Recreation    0.069396
Shopping      0.056772
Reference     0.048541
Health        0.045197
Games         0.022161
Home          0.015794
News          0.008062
Name: topic_main, dtype: float64


In [49]:
train=pd.concat([X_resampled,pd.get_dummies(X_resampled['topic_main'])],axis=1)
train.sample(5)

,externalpage,title,description,catid,topic_main,Arts,Business,Computers,Games,Health,Home,News,Recreation,Reference,Science,Shopping,Society,Sports
872546,http://melinda-horses-r-art.pixels.com/,Horses R Art,Features the digital inspired work of Melinda ...,968565,Shopping,0,0,0,0,0,0,0,0,0,0,1,0,0
469146,http://www.mtr-ltd.com/,Materials Technologies Research,Offers a wide variety of Fullerene Products.,273102,Business,0,1,0,0,0,0,0,0,0,0,0,0,0
328466,http://www.abcsheetmetal.com/,ABC Sheet Metal,Offers broad range of metal fabrication capabi...,277438,Business,0,1,0,0,0,0,0,0,0,0,0,0,0
1623386,https://www.facebook.com/The-Monroe-Street-Jou...,Monroe Street Journal,Weekly newspaper of the University of Michigan...,998841,News,0,0,0,0,0,0,1,0,0,0,0,0,0
961475,http://www.tdt-ok.com/,Central Tech,Truck driver training school in Oklahoma that ...,275284,Business,0,1,0,0,0,0,0,0,0,0,0,0,0


In [50]:
del df

In [51]:
val=train

In [52]:
num_labels=y_ros.nunique()

#data={'catid':[0],
#      'topic':[text],
#}
#test=pd.DataFrame(data)
#for labels in df_label['topic_main'].unique():
  # label='label_'+str(labels)
  # label_cols.append(label)
#  test[labels]=0

#test=test.append(test)

In [53]:
if config.testing:
    train = train.head(1024)
    val = val.head(1024)
    # test = test.head(1024)

In [54]:
fastai_bert_vocab = Vocab(list(bert_tok.vocab.keys()))

In [55]:
fastai_tokenizer = Tokenizer(tok_func=FastAiBertTokenizer(bert_tok, max_seq_len=config.max_seq_len), pre_rules=[], post_rules=[])

In [56]:
label_cols=y_ros.unique().tolist()
print(label_cols)

['Society', 'Recreation', 'Science', 'Sports', 'Business', 'Home', 'Reference', 'Computers', 'Arts', 'Shopping', 'News', 'Health', 'Games']


In [57]:
label_cols[0]

'Society'

In [58]:
class BertTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
        super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class BertNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=Vocab(list(bert_tok.vocab.keys())), **kwargs)

def get_bert_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for BERT
    We remove sos/eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original BERT model.
    """
    return [BertTokenizeProcessor(tokenizer=tokenizer),NumericalizeProcessor(vocab=vocab)]

In [59]:
class BertDataBunch(TextDataBunch):
    @classmethod
    def from_df(cls, path:PathOrStr, train_df:DataFrame, valid_df:DataFrame, test_df:Optional[DataFrame]=None,
                tokenizer:Tokenizer=None, vocab:Vocab=None, classes:Collection[str]=None, text_cols:IntsOrStrs=1,
                label_cols:IntsOrStrs=0, label_delim:str=None, **kwargs) -> DataBunch:
        "Create a `TextDataBunch` from DataFrames."
        p_kwargs, kwargs = split_kwargs_by_func(kwargs, get_bert_processor)
        # use our custom processors while taking tokenizer and vocab as kwargs
        processor = get_bert_processor(tokenizer=tokenizer, vocab=vocab, **p_kwargs)
        if classes is None and is_listy(label_cols) and len(label_cols) > 1: classes = label_cols
        src = ItemLists(path, TextList.from_df(train_df, path, cols=text_cols, processor=processor),
                        TextList.from_df(valid_df, path, cols=text_cols, processor=processor))
        src = src.label_for_lm() if cls==TextLMDataBunch else src.label_from_df(cols=label_cols, classes=classes)
        if test_df is not None: src.add_test(TextList.from_df(test_df, path, cols=text_cols))
        return src.databunch(**kwargs)

In [60]:
databunch = BertDataBunch.from_df(".", train, val, None,
                  tokenizer=fastai_tokenizer,
                  vocab=fastai_bert_vocab,
                  text_cols="topic_main",
                  label_cols=label_cols,
                  bs=config.bs,
                  collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
             )

In [61]:
from pytorch_pretrained_bert.modeling import BertConfig, BertForSequenceClassification
bert_model = BertForSequenceClassification.from_pretrained(config.bert_model_name, num_labels=num_labels)

In [62]:
loss_func = nn.BCEWithLogitsLoss()

In [63]:
from fastai.callbacks import *
device_cuda = torch.device("cuda")
learner = Learner(
    databunch, bert_model,
    loss_func=loss_func
)
learner.model.cuda()
if config.use_fp16: learner = learner.to_fp16()
# learner.to_fp32()

In [65]:
learner.lr_find()

epoch,train_loss,valid_loss,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: 

In [ ]:
learner.recorder.plot()

In [ ]:
# learner.fit_one_cycle(config.epochs, max_lr=config.max_lr)
# learner.fit_one_cycle(config.epochs, max_lr=1e-02)
learner.fit_one_cycle(8, max_lr=config.max_lr)

epoch,train_loss,valid_loss,time
0,0.000000,0.000000,2:46:54
1,0.000000,0.000000,2:42:00
2,0.000000,0.000000,2:38:54


Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


In [ ]:
train.catid.nunique()

In [ ]:
learner.export(file='/kaggle/working/dmoz_model.pkl')

In [ ]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    """
    the get_preds method does not yield the elements in order by default
    we borrow the code from the RNNLearner to resort the elements into their correct order
    """
    preds = learner.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in databunch.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    return preds[reverse_sampler, :]

In [ ]:
# learner.export(file=DATA_ROOT /"dmoz_model.pkl")
# learner.save(file=DATA_ROOT/'dmoz_model.pkl')

In [ ]:
learner.validate()

In [ ]:
text='Efforts were being made on Friday to help passengers stranded at the airport, railway station and inter-state bus terminals in Guwahati, an official said.'

In [ ]:
# learner.load(file=DATA_ROOT/'dmoz_model.pkl')
x=learner.predict(text)
y=pd.DataFrame(x[2],index=label_cols)
# x[2].shape
y.nlargest(5,[0])